# Using OnnxRuntime with instructor-base
- Run `optimum-cli export onnx --model hkunlp/instructor-base instructor_base_onnx/`

### Load onnx model

In [2]:
from onnxruntime import InferenceSession
import numpy as np


# only need encoder
instructor_encoder_sess = InferenceSession('./instructor_base_onnx/encoder_model.onnx')

### Load Instructor model

In [3]:
from InstructorEmbedding import INSTRUCTOR

# OG model
instructor_model = INSTRUCTOR('hkunlp/instructor-base')

# onnx test model
instructor_onnx = INSTRUCTOR('hkunlp/instructor-base')

/Users/mrezanvari/_MyFiles/Programs/Patagona-technologies/ggml-python/examples/instructor/.venv/lib/python3.8/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512


### Custom onnx model

In [5]:
import torch.nn as nn

class OnnxModule(nn.Module):
    def __init__(self, inference_session):
          super().__init__()
          self._inference_session = inference_session
          
    def forward(self, *args, **kwargs):

          kwargs = {'input_ids': np.array(kwargs['input_ids']), 'attention_mask': np.array(kwargs['attention_mask'])}
          return self._inference_session.run(None, kwargs)


# replace transfromer model forward call with custom onnx model

for i in instructor_onnx.modules():
    if i._get_name() == 'INSTRUCTOR_Transformer':
        i.auto_model.__call__ = OnnxModule(instructor_encoder_sess).forward
        
        break

### Compare embeddings

In [6]:
sentence = "This is a sentence to test instructor embedding model"
instruction = ""


real_embeddings = instructor_model.encode([[instruction, sentence]])
onnx_embeddings = instructor_onnx.encode([[instruction, sentence]])

In [7]:
from sklearn.metrics.pairwise import cosine_similarity


cosine_similarity(onnx_embeddings, real_embeddings)


array([[1.]], dtype=float32)